In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, precision_recall_curve
from imblearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix
import time # to measure how long the models take
from sklearn import datasets
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, precision_recall_curve

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV

In [20]:
df=pd.read_csv('/Users/dylanbai/Desktop/Tableau/assignment/Dinesafe.csv')

In [21]:
df.head()

,_id,Rec #,Establishment ID,Inspection ID,Establishment Name,Establishment Type,Establishment Address,Establishment Status,Min. Inspections Per Year,Infraction Details,Inspection Date,Severity,Action,Outcome,Amount Fined,Latitude,Longitude
0,1,1,10657713,104953972.0,NEW KANTAMANTO MARKET,Food Depot,"266 EDDYSTONE AVE, Unit-0",Pass,2,NaN,2022-04-05,NaN,NaN,NaN,NaN,43.74791,-79.52219
1,2,2,10657713,105133203.0,NEW KANTAMANTO MARKET,Food Depot,"266 EDDYSTONE AVE, Unit-0",Pass,2,Operate food premise - equipment not arranged ...,2023-03-07,M - Minor,Notice to Comply,NaN,NaN,43.74791,-79.52219
2,3,3,10657713,105133203.0,NEW KANTAMANTO MARKET,Food Depot,"266 EDDYSTONE AVE, Unit-0",Pass,2,FOOD PREMISE NOT MAINTAINED WITH CLEAN FLOORS ...,2023-03-07,M - Minor,Notice to Comply,NaN,NaN,43.74791,-79.52219
3,4,4,10657713,105238109.0,NEW KANTAMANTO MARKET,Food Depot,"266 EDDYSTONE AVE, Unit-0",Pass,2,NaN,2023-08-25,NaN,NaN,NaN,NaN,43.74791,-79.52219
4,5,5,10752656,104965345.0,# HASHTAG INDIA RESTAURANT,Food Take Out,1871 O'CONNOR DR,Pass,3,FOOD PREMISE NOT MAINTAINED WITH FOOD HANDLING...,2022-04-29,M - Minor,Notice to Comply,NaN,NaN,43.72199,-79.30349


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70318 entries, 0 to 70317
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   _id                        70318 non-null  int64  
 1   Rec #                      70318 non-null  int64  
 2   Establishment ID           70318 non-null  int64  
 3   Inspection ID              69841 non-null  float64
 4   Establishment Name         70318 non-null  object 
 5   Establishment Type         70318 non-null  object 
 6   Establishment Address      70318 non-null  object 
 7   Establishment Status       70318 non-null  object 
 8   Min. Inspections Per Year  70318 non-null  object 
 9   Infraction Details         43737 non-null  object 
 10  Inspection Date            69841 non-null  object 
 11  Severity                   43737 non-null  object 
 12  Action                     43737 non-null  object 
 13  Outcome                    172 non-null    obj

In [23]:
df.describe()

,_id,Rec #,Establishment ID,Inspection ID,Amount Fined,Latitude,Longitude
count,70318.000000,70318.000000,7.031800e+04,6.984100e+04,111.000000,70318.000000,70318.000000
mean,35159.500000,35159.500000,1.038849e+07,1.051048e+08,482.459459,43.699189,-79.397929
std,20299.202451,20299.202451,5.757684e+05,1.009361e+05,628.953009,0.053144,0.083077
min,1.000000,1.000000,1.222579e+06,1.042890e+08,0.000000,43.587910,-79.634550
25%,17580.250000,17580.250000,1.039244e+07,1.050228e+08,75.000000,43.654880,-79.442910
50%,35159.500000,35159.500000,1.059546e+07,1.051150e+08,385.000000,43.686780,-79.398320
75%,52738.750000,52738.750000,1.072658e+07,1.051884e+08,580.000000,43.739740,-79.350820
max,70318.000000,70318.000000,1.079647e+07,1.052696e+08,3755.000000,43.839140,-79.130690


In [32]:
df = df.drop('Outcome',axis=1)
df= df.drop('Amount Fined',axis=1)

In [33]:
df_cleaned = df.dropna()


In [34]:
unique_values = df_cleaned['Establishment Status'].unique()
print("Unique values in Establishment Status:", unique_values)

Unique values in Establishment Status: ['Pass' 'Conditional Pass']


In [35]:
unique_values = df_cleaned['Severity'].unique()
print("Unique values in Severity:", unique_values)

Unique values in Severity: ['M - Minor' 'C - Crucial' 'S - Significant' 'NA - Not Applicable']


In [36]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43737 entries, 1 to 70317
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   _id                        43737 non-null  int64  
 1   Rec #                      43737 non-null  int64  
 2   Establishment ID           43737 non-null  int64  
 3   Inspection ID              43737 non-null  float64
 4   Establishment Name         43737 non-null  object 
 5   Establishment Type         43737 non-null  object 
 6   Establishment Address      43737 non-null  object 
 7   Establishment Status       43737 non-null  object 
 8   Min. Inspections Per Year  43737 non-null  object 
 9   Infraction Details         43737 non-null  object 
 10  Inspection Date            43737 non-null  object 
 11  Severity                   43737 non-null  object 
 12  Action                     43737 non-null  object 
 13  Latitude                   43737 non-null  float64


In [41]:
unique_count = df_cleaned['Establishment Name'].nunique()
print("Number of unique values in Establishment Name:", unique_count)

Number of unique values in Establishment Name: 7839


In [49]:
unique_values = df_cleaned['Min. Inspections Per Year'].unique()
print("Min. Inspections Per Year:", unique_values)

Min. Inspections Per Year: ['2' '3' '1' 'O']


In [50]:
count_rows_with_specific_value = len(df_cleaned[df_cleaned['Min. Inspections Per Year'] == '0'])
print(count_rows_with_specific_value)

0


In [52]:
unique_count = df_cleaned['Latitude'].nunique()
print("Number of unique values in Latitude:", unique_count)

Number of unique values in Latitude: 5502


In [51]:
df_cleaned.to_csv('/Users/dylanbai/Desktop/Tableau/assignment/cleaned_data.csv', index=False)